# Find the comparables: real_acc.txt

The file `real_acc.txt` contains important property information like number total appraised value (the target on this exercise), neighborhood, school district, economic group, land value, and more. Let's load this file and grab a subset with the important columns to continue our study.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import pandas as pd

from src.definitions import ROOT_DIR
from src.data.utils import Table, save_pickle

In [3]:
real_acct_fn = ROOT_DIR / 'data/external/2016/Real_acct_owner/real_acct.txt'
assert real_acct_fn.exists()

In [4]:
real_acct = Table(real_acct_fn, '2016')

In [5]:
real_acct_df = real_acct.get_df()

# Load accounts of interest
Let's remove the account numbers that don't meet free-standing single-family home criteria that we found while processing the `building_res.txt` file.

In [6]:
one_bld_in_acct_fn = ROOT_DIR / 'data/raw/2016/one_bld_in_acct.pickle'

In [7]:
with open(one_bld_in_acct_fn, 'rb') as f:
    one_bld_in_acct = pickle.load(f)

In [8]:
cond0 = real_acct_df['acct'].isin(one_bld_in_acct)
real_acct_df = real_acct_df.loc[cond0, :]

In [9]:
real_acct_df.head()

,acct,yr,mailto,mail_addr_1,mail_addr_2,mail_city,mail_state,mail_zip,mail_country,undeliverable,...,protested,certified_date,rev_dt,rev_by,new_own_dt,lgl_1,lgl_2,lgl_3,lgl_4,jurs
1147,21440000001,2016,TREWICK MICHAEL & MEGONE,2120 LIVE OAK ST,,HOUSTON,TX,77003-6026,,N,...,N,2016-08-12 00:00:00.000,2016-05-11 11:46:44.107,01548,2012-09-11 00:00:00.000,LT 1 BLK 426,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 944
1181,21470000008,2016,COHEN JAY H % TOWMEY MAY PLLC,2 RIVERWAY FL 15,,HOUSTON,TX,77056-1939,,N,...,Y,2016-08-12 00:00:00.000,2013-12-16 12:22:41.343,01327,1988-01-02 00:00:00.000,LT 8 BLK 429,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 888 944
1188,21480000002,2016,DOUGLAS DELOIS L DOUGLAS FENWICK ESTATE OF,2116 HUTCHINS ST,,HOUSTON,TX,77003-5934,,N,...,N,2016-08-12 00:00:00.000,2013-12-16 12:13:21.600,01327,2004-07-28 00:00:00.000,LT 2 BLK 430,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 888 944
1235,21650000007,2016,SUAREZ JORGE A,2217 HUTCHINS ST,,HOUSTON,TX,77003-5935,,N,...,Y,2016-05-17 00:00:00.000,2015-12-02 14:38:09.433,01217,2013-10-14 00:00:00.000,LT 7 BLK 447,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 944
1236,21650000011,2016,BROOKS LONNIE & LISA,2212 WEBSTER ST,,HOUSTON,TX,77003-5946,,N,...,N,2016-08-12 00:00:00.000,2013-12-13 11:00:19.397,01327,2001-05-05 00:00:00.000,LT 11 BLK 447,SSBB,NaN,NaN,001 040 041 042 043 044 048 061 944


In [10]:
real_acct_df.columns

Index(['acct', 'yr', 'mailto', 'mail_addr_1', 'mail_addr_2', 'mail_city',
       'mail_state', 'mail_zip', 'mail_country', 'undeliverable', 'str_pfx',
       'str_num', 'str_num_sfx', 'str', 'str_sfx', 'str_sfx_dir', 'str_unit',
       'site_addr_1', 'site_addr_2', 'site_addr_3', 'state_class',
       'school_dist', 'map_facet', 'key_map', 'Neighborhood_Code',
       'Neighborhood_Grp', 'Market_Area_1', 'Market_Area_1_Dscr',
       'Market_Area_2', 'Market_Area_2_Dscr', 'econ_area', 'econ_bld_class',
       'center_code', 'yr_impr', 'yr_annexed', 'splt_dt', 'dsc_cd', 'nxt_bld',
       'bld_ar', 'land_ar', 'acreage', 'Cap_acct', 'shared_cad', 'land_val',
       'bld_val', 'x_features_val', 'ag_val', 'assessed_val', 'tot_appr_val',
       'tot_mkt_val', 'prior_land_val', 'prior_bld_val',
       'prior_x_features_val', 'prior_ag_val', 'prior_tot_appr_val',
       'prior_tot_mkt_val', 'new_construction_val', 'tot_rcn_val',
       'value_status', 'noticed', 'notice_dt', 'protested', 'certif

# Select columns and build pivot table
The columns above show a lot of value information along property groups that might come in handy when predicting the appraised value. Now let's get a slice of some of the important columns.

In [11]:
cols = [
    'acct',
    'site_addr_3', # Zip
    'school_dist',
    'Neighborhood_Code',
    'Market_Area_1_Dscr',
    'Market_Area_2_Dscr',
    'center_code',
    'bld_ar',
    'land_ar',
    'acreage',
    'land_val',
    'tot_appr_val', # Target
    'prior_land_val',
    'prior_tot_appr_val',
    'new_own_dt',  # New owner date
]

In [12]:
real_acct_df = real_acct_df.loc[:, cols]

In [13]:
real_acct_df.head()

,acct,site_addr_3,school_dist,Neighborhood_Code,Market_Area_1_Dscr,Market_Area_2_Dscr,center_code,bld_ar,land_ar,acreage,land_val,tot_appr_val,prior_land_val,prior_tot_appr_val,new_own_dt
1147,21440000001,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,2537,5000,0.1148,125000.0,145200.0,75000.0,132000.0,2012-09-11 00:00:00.000
1181,21470000008,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1000,5000,0.1148,74900.0,75000.0,64000.0,65000.0,1988-01-02 00:00:00.000
1188,21480000002,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1496,5000,0.1148,125000.0,85929.0,75000.0,78118.0,2004-07-28 00:00:00.000
1235,21650000007,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,3387,5000,0.1148,125000.0,549004.0,75000.0,75000.0,2013-10-14 00:00:00.000
1236,21650000011,77003,01,8400.12,"1C Midtown, Riverside Terrace, University Areas","1C Midtown, Riverside Terrace, University Areas",61,1508,6250,0.1435,140625.0,181370.0,84375.0,164882.0,2001-05-05 00:00:00.000


Double check if the there is only one account number per row

In [14]:
assert real_acct_df['acct'].is_unique

# Export real_acct

In [15]:
save_fn = ROOT_DIR / 'data/raw/2016/real_acct_comps.pickle'
save_pickle(real_acct_df, save_fn)